# Introduction and Usage Guidelines

This notebook is designed as a base for analysis for Trackmania's physics. 

You can view API documentation here: http://localhost:8080/swagger-ui/index.html#, in the 'out' controller section. 

(The 'in' section is what API is used by the GigaChad Collector (GCC) from within Trackmania.)

In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

# optional - makes plots pop out. highly recommended
# %matplotlib qt

# no trailing slash
url_prod = "http://gigachad.justinjschmitz.com:21532";
url_dev = "http://localhost:8080";

SERVER_PATH = url_prod

SWAGGER_PATH = f"{SERVER_PATH}/swagger-ui/index.html"

print(f"Current documentation page: {SWAGGER_PATH}")


Current documentation page: http://gigachad.justinjschmitz.com:21532/swagger-ui/index.html


## Data Selection

### "Maptags" vs "Runkeys" 

This is kind of an arbitrary convention, but:
* Map tags are for general surface use. 
* Runkeys are for specific study use. 

E.g., if you wanted to just look at how the stadium car drives on dirt, then you can pull the map tag 'stadium - dirt' and you'll have all that data. 

But, if you were doing some novel study (like trying to 2 wheel the rally car, or something) and wanted *just* that data, then make sure you set that as the the underlying runkey. That way, you're not pulling in more than you need to. 



In [2]:
maptags = requests.get(f"{SERVER_PATH}/out/maptag")
runkeys = requests.get(f"{SERVER_PATH}/out/runkey")
maptags_df = pd.json_normalize(maptags.json())
runkeys_df = pd.json_normalize(runkeys.json())
display(maptags_df)
display(runkeys_df)

# change these values with 'tag' and 'name' respectively

SELECTED_MAPTAG = 'stadium - dirt'
SELECTED_RUNKEY = 'test2'


,mtId,mapUuid,tag,username
0,2,bu5OUM_XDFQ7w_vnhQOopNsyJQ8,dirt - stadium,sgt_bigbird
1,3,nDqOzuoWkPUJ9m74q0ihDVlTFn7,dirt - snow,sgt_bigbird
2,4,OuHxc71KtYLTkb7r6AmJcHCbbz8,dirt - rally,sgt_bigbird
3,5,M92iB3BWeAsGChHVL6QqqtAnNC6,dirt - desert,sgt_bigbird


,rkId,name,mode
0,102,default,1


In [3]:
# For 'maptag' data:
maptag_data = requests.get(f"{SERVER_PATH}/out/data/tag", params={"tag": SELECTED_MAPTAG})
# For 'runkey' data:
runkey_data = requests.get(f"{SERVER_PATH}/out/data/runkey", params={"runkey": SELECTED_RUNKEY})

print(f"Loaded {len(maptag_data.text)} chars of maptag data, {len(runkey_data.text)} chars of runkey data")

Loaded 113 chars of maptag data, 116 chars of runkey data


In [4]:
data = maptag_data
# data = runkey_data
df = pd.json_normalize(data.json())

In [5]:
# Example: Graph of acceleration at different slide angles

## Preprocess: 

# Acceleration isn't natively provided - do .diff() for this and any other derivatives you want to calculate
# You can do a rolling average, too, depending on usage
df["acc"] = df["speed"].diff()

## Filter:

df = df[abs(df["acc"]) < 0.5]

## Now scatter the data 
plt.scatter(x=df["slipDir"], y=df["acc"], c=df["speed"], alpha=1)
plt.show()

KeyError: 'speed'